In [33]:
from ioh import get_problem, ProblemClass
from ioh import logger
import sys
import numpy as np
import time

In [34]:
# Declaration of problems to be tested.
# We obtain an interface of the OneMax problem here.
dimension = 50

In [35]:
"""
1 (fid) : The funciton ID of the problem in the problem suite. OneMax is 1 defined within the PBO class. 2 would correspond to another problem.
dimension : The dimension of the problem, which we have set to 50.
instance: In benchmarking libraries, problems often have multiple instances. These instances may vary slightly (e.g., different random noise, shifts, etc.) 
            to allow algorithms to be tested on a variety of conditions.
om(x) return the fitness value of 'x'
"""
om = get_problem(1, dimension=dimension, instance=1, problem_class=ProblemClass.PBO)
# We know the optimum of onemax
optimum = dimension

In [36]:
# Create default logger compatible with IOHanalyzer
# `root` indicates where the output files are stored.
# `folder_name` is the name of the folder containing all output. You should compress the folder 'run' and upload it to IOHanalyzer.
l = logger.Analyzer(root="data", 
    folder_name="run", 
    algorithm_name="genetic_algorithm", 
    algorithm_info="The lab session of the evolutionary algorithm course in LIACS")

In [37]:
om.attach_logger(l)

In [38]:
# Parameters setting
pop_size = 10
tournament_k = 5
mutation_rate = 0.03
crossover_probability = 0.5

In [39]:
# Uniform Crossover
def crossover(p1, p2):
    if np.random.uniform(0, 1)<crossover_probability:
        cross_point = np.random.randint(0, 2, len(p1)).astype(np.bool_)
        p1[cross_point] = p2[cross_point]
    '''for i in range(len(p1)):
        if np.random.rand() < crossover_probability:
            p1[i] = p2[i]'''
    return p1

In [40]:
# Standard bit mutation using mutation rate p
def mutation(p):
    for mutation_point in range(len(p)):
        if np.random.rand() < mutation_rate:
            p[mutation_point] = 1 if p[mutation_point] == 0 else 0
    return p


selection_tourament

In [41]:
def mating_seletion(parent, parent_f):
    parents = []
    for _ in range(len(parent)):
        parent_index =  np.random.choice(len(parent), size = tournament_k, replace=False)
        parent = parent[parent_index]
        parent_f = parent_f[parent_index]
        new_parent = parent[np.argmax(parent_f)]
        parents.append(new_parent)
    return np.array(parents)

rank selection

In [42]:
'''def mating_seletion(parent, parent_f):
    rank = np.argsort(parent_f)
    probabilities = np.arange(1, len(parent_f) + 1) / sum(range(1, len(parent_f) + 1))
    # choose with probability of rank
    selected_indices = np.random.choice(rank, size=len(parent), p=probabilities)
    parents = parent[selected_indices]
    return np.array(parents)'''

'def mating_seletion(parent, parent_f):\n    rank = np.argsort(parent_f)\n    probabilities = np.arange(1, len(parent_f) + 1) / sum(range(1, len(parent_f) + 1))\n    # choose with probability of rank\n    selected_indices = np.random.choice(rank, size=len(parent), p=probabilities)\n    parents = parent[selected_indices]\n    return np.array(parents)'

SUS

In [43]:
'''def mating_seletion(parent, parent_f) :
    fitness = np.sum(parent_f)
    point_distance = fitness / len(parent)
    start_point = np.random.uniform(0, point_distance)
    points = [start_point + i * point_distance for i in range(len(parent))]
    selected = []
    for point in points:
        i = 0
        while True:
            point -= parent_f[i]
            if point > parent_f[i]:
                i += 1
            else:
                selected.append(parent[i])
                break
    return np.array(selected)'''

'def mating_seletion(parent, parent_f) :\n    fitness = np.sum(parent_f)\n    point_distance = fitness / len(parent)\n    start_point = np.random.uniform(0, point_distance)\n    points = [start_point + i * point_distance for i in range(len(parent))]\n    selected = []\n    for point in points:\n        i = 0\n        while True:\n            point -= parent_f[i]\n            if point > parent_f[i]:\n                i += 1\n            else:\n                selected.append(parent[i])\n                break\n    return np.array(selected)'

Roulette

In [44]:
# Using the Fitness proportional selection
# Roulette
'''def mating_seletion(parent, parent_f) :
    fitness = np.sum(parent_f)
    p = parent_f/fitness
    sum = [0]
    for i in range(len(p)):
        s = 0
        for j in range(i+1):
            s += p[j] 
        sum.append(s)
    parents = []
    for _ in range(pop_size):
        roll = np.random.uniform(0,1)
        for i in range(1, len(p)):
            if roll>sum[i] and roll<sum[i+1]:
                parents.append(parent[i])
                break
            else:
                parents.append(parent[0])
                break
    
    return np.array(parents)'''

'def mating_seletion(parent, parent_f) :\n    fitness = np.sum(parent_f)\n    p = parent_f/fitness\n    sum = [0]\n    for i in range(len(p)):\n        s = 0\n        for j in range(i+1):\n            s += p[j] \n        sum.append(s)\n    parents = []\n    for _ in range(pop_size):\n        roll = np.random.uniform(0,1)\n        for i in range(1, len(p)):\n            if roll>sum[i] and roll<sum[i+1]:\n                parents.append(parent[i])\n                break\n            else:\n                parents.append(parent[0])\n                break\n    \n    return np.array(parents)'

random_selection

In [45]:
'''def mating_seletion(parent, parent_f):
    indices = np.random.choice(len(parent), len(parent), replace=True)
    return parent[indices]'''

'def mating_seletion(parent, parent_f):\n    indices = np.random.choice(len(parent), len(parent), replace=True)\n    return parent[indices]'

In [46]:
def genetic_algorithm(func, budget = None):
    
    # budget of each run: 10000
    if budget is None:
        budget = 10000
    
    # f_opt : Optimal function value.
    # x_opt : Optimal solution.
    f_opt = sys.float_info.min
    x_opt = None
    
    # parent : A list that holds the binary strings representing potential solutions or individuals in the current population.
    # parent_f : A list that holds the fitness values corresponding to each individual in the parent list.
    parent = []
    parent_f = []
    for i in range(pop_size):
        # Initialization
        parent.append(np.random.randint(2, size = func.meta_data.n_variables))
        parent_f.append(func(parent[i]))
        budget = budget - 1
    
    while (f_opt < optimum and budget > 0):
        parent = np.array(parent)
        parent_f = np.array(parent_f)
        
        parent = mating_seletion(parent, parent_f)
        parent_f = np.zeros_like(parent)
        print(parent.shape)
        for i in range(len(parent)):
            parent_f[i] = func(parent[i])

        
        # Perform mating selection, crossover, and mutation to generate offspring
        '''parent_index =  np.random.choice(len(parent), size = tournament_k, replace=False)
        parent = parent[parent_index]
        parent_f = parent_f[parent_index]'''

        new_parent = []
        new_parent_f = []
        for i in range(len(parent)):
            for j in range(i+1, len(parent)):
                offspring = mutation(crossover(parent[i], parent[j]))
                new_parent.append(offspring)
                new_parent_f.append(func(offspring))
                
                '''parent = np.vstack((parent, offspring))
                parent_f = np.insert(parent_f, len(parent_f), func(offspring))'''
                budget = budget - 1
        
        for i in range(len(new_parent)):
            if func(new_parent[i])>f_opt:
                f_opt = func(new_parent[i])
                x_opt = new_parent[i]
        
        new_parent_f = np.array(new_parent_f)
        fitness_sort = np.argsort(new_parent_f)[::-1]
        new_parent = np.array(new_parent)
        parent = new_parent[fitness_sort][:pop_size]
        parent_f = new_parent_f[fitness_sort][:pop_size]


    
    # ioh function, to reset the recording status of the function.
    func.reset()
    print(f_opt,x_opt)
    return f_opt, x_opt

In [47]:
def main():
    # We run the algorithm 20 independent times.
    for _ in range(20):
        genetic_algorithm(om)

In [48]:
if __name__ == '__main__':
  start = time.time()
  main()
  end = time.time()
  print("The program takes %s seconds" % (end-start))

(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(10, 50)
(

KeyboardInterrupt: 